In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
%cd drive/MyDrive/nlp_project

/content/drive/MyDrive/nlp_project


In [ ]:
!pip install arabic-stopwords
!pip install qalsadi
!pip install pyarabic
!pip install gensim
!pip install top2vec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import numpy as np
import pandas as pd
import spacy
import string
import qalsadi.lemmatizer
from nltk.stem.isri import ISRIStemmer
from pyarabic.araby import tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn import metrics
from sklearn.svm import LinearSVC, SVC
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB, ComplementNB
import torch
from torch import nn
from tqdm import tqdm
import pickle

from gru_model import ArabicDataset, Classifier, evaluate, train
from pre_processing_post import processPost
from feature_extraction import get_ngram_features, get_word_embedding_features, avg_word_vector, get_word_count_features

In [2]:
# needed functions
def print_report(pipe, x_test, y_test):
    y_pred = pipe.predict(x_test)
    report = metrics.classification_report(y_test, y_pred)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

# Read train data

In [3]:
train_data = pd.read_csv('./DataSet/train.csv',sep=',',header=0)
test_data = pd.read_csv('./DataSet/dev.csv',sep=',',header=0)
train_data.head(10)

,text,category,stance
0,بيل غيتس يتلقى لقاح #كوفيد19 من غير تصوير الاب...,celebrity,1
1,وزير الصحة لحد اليوم وتحديدا هلأ بمؤتمروا الصح...,info_news,1
2,قولكن رح يكونو اد المسؤولية ب لبنان لما يوصل ...,info_news,1
3,#تركيا.. وزير الصحة فخر الدين قوجة يتلقى أول ج...,celebrity,1
4,وئام وهاب يشتم الدول الخليجية في كل طلة اعلامي...,personal,0
5,"لقاح #كورونا في أميركا.. قلق متزايد من ""التوزي...",info_news,0
6,لبنان اشترى مليونان لقاح امريكي اذا شلنا يلي ع...,info_news,1
7,من عوارض لقاح كورونا<LF>هو تهكير حسابك عتويتر<...,personal,0
8,هناك 1780 مليونيراً في لبنان. ماذا لو فُرضت ال...,unrelated,0
9,دعبول حضرتك منو انت وتطلب من قائد دولة إسلامية...,info_news,1


In [4]:
#remove first row that has the header
train_data['category'] = train_data['category'].astype('category').cat.codes
train_data.head(10)

,text,category,stance
0,بيل غيتس يتلقى لقاح #كوفيد19 من غير تصوير الاب...,1,1
1,وزير الصحة لحد اليوم وتحديدا هلأ بمؤتمروا الصح...,2,1
2,قولكن رح يكونو اد المسؤولية ب لبنان لما يوصل ...,2,1
3,#تركيا.. وزير الصحة فخر الدين قوجة يتلقى أول ج...,1,1
4,وئام وهاب يشتم الدول الخليجية في كل طلة اعلامي...,4,0
5,"لقاح #كورونا في أميركا.. قلق متزايد من ""التوزي...",2,0
6,لبنان اشترى مليونان لقاح امريكي اذا شلنا يلي ع...,2,1
7,من عوارض لقاح كورونا<LF>هو تهكير حسابك عتويتر<...,4,0
8,هناك 1780 مليونيراً في لبنان. ماذا لو فُرضت ال...,9,0
9,دعبول حضرتك منو انت وتطلب من قائد دولة إسلامية...,2,1


## Over Sampling

In [5]:
# !pip install imblearn

In [6]:
# from collections import Counter
# from imblearn.over_sampling import RandomOverSampler
# train_data=train_data.drop('category',axis=1)
# y=train_data['stance']
# print(Counter(train_data['stance']))
# train_data=train_data.drop('stance',axis=1)
# # define oversampling strategy
# oversample = RandomOverSampler(random_state=3)
# # fit and apply the transform
# train_data["text"], train_data['stance'] = oversample.fit_resample(train_data, y)
# print(Counter(train_data['stance']))

# Pre-Processing the tweets

In [7]:
print(train_data["text"][9])
train_data["text"] = train_data['text'].apply(lambda x: processPost(x))
test_data['text'] = test_data['text'].apply(lambda x: processPost(x))
print(train_data["text"][9])

دعبول حضرتك منو انت وتطلب من قائد دولة إسلامية لقاح لعد ما اتابع الاخبار هم بكل مجالاتهم متفوقين وراح يطلع اللقاح قريباً؟<LF>#دعبول_دومه_مسحول
دعبول حضر من نت طلب قائد دول إسلام قاح عد تابع اخبار مجال متفوق طلع قاح قريبا دعبول دوم مسحول


## Ara2Vec Embedding

In [8]:
# load AraVec Spacy model
nlp = spacy.load("./spacy.aravec.model/")

c:\Users\Nada\anaconda3\lib\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'ar_pipeline' (0.0.0) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [9]:
# Define the preprocessing Class
class Preprocessor:
    def __init__(self, tokenizer, **cfg):
        self.tokenizer = tokenizer

    def __call__(self, text):
        return self.tokenizer(text)

In [10]:
# Apply the `Preprocessor` Class
nlp.tokenizer = Preprocessor(nlp.tokenizer)

## create vocablary

In [11]:
train_data_tokenized = train_data['text'].apply(tokenize)
test_data_tokenized = test_data['text'].apply(tokenize)
#merge all the sentences in one list
vocab = [item for sublist in train_data_tokenized for item in sublist]
vocab = list(set(vocab))
vocab.append('<فراغ>') 
vocab.insert(0, '<مجهول>') 
word2index = {word: i for i, word in enumerate(vocab)}

In [12]:
# save the word2index dictionary
with open('./vocab/stance/word2index.pickle', 'wb') as handle:
    pickle.dump(word2index, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Feature Extraction

## 1. TD-IDF

In [13]:
ngramdata_features, word_vectorizer = get_ngram_features(train_data)
ngramdata_features.head()

# save the model to disk
filename = './models/stance/TFIDFVectorizer.sav'
pickle.dump(word_vectorizer, open(filename, 'wb'))

LinearSVC Classifier

In [39]:
clf = SVC(kernel='linear', class_weight="balanced")
pipe_tfidf = make_pipeline(word_vectorizer, clf)
pipe_tfidf.fit(train_data['text'], train_data['stance'])
print_report(pipe_tfidf, test_data['text'], test_data['stance'])

# save the model to disk
filename = './models/stance/LinearSVC_tfidf.sav'
pickle.dump(pipe_tfidf, open(filename, 'wb'))

              precision    recall  f1-score   support

          -1       0.23      0.59      0.33        70
           0       0.29      0.58      0.38       126
           1       0.96      0.67      0.79       804

    accuracy                           0.66      1000
   macro avg       0.49      0.61      0.50      1000
weighted avg       0.82      0.66      0.71      1000

accuracy: 0.656


RandomForest Classifier

In [15]:
X_train_tfidf = word_vectorizer.fit_transform(train_data['text'])
X_test_tfidf = word_vectorizer.transform(test_data['text'])
rf = RandomForestClassifier(class_weight="balanced")
rf_tfidf = rf.fit(X_train_tfidf, train_data['stance'])
y_pred = rf_tfidf.predict(X_test_tfidf)

print_report(rf_tfidf, X_test_tfidf, test_data['stance'])

# save the model to disk
filename = './models/stance/RandomForest_tfidf.sav'
pickle.dump(rf_tfidf, open(filename, 'wb'))

              precision    recall  f1-score   support

          -1       0.33      0.16      0.21        70
           0       0.45      0.28      0.34       126
           1       0.85      0.94      0.89       804

    accuracy                           0.80      1000
   macro avg       0.54      0.46      0.48      1000
weighted avg       0.76      0.80      0.78      1000

accuracy: 0.801


naive bayes

In [16]:
X_train_tfidf = word_vectorizer.fit_transform(train_data['text'])
X_test_tfidf = word_vectorizer.transform(test_data['text'])
CNB_clr = ComplementNB()
CNB_clr_tfidf = CNB_clr.fit(X_train_tfidf, train_data['stance'])
print_report(CNB_clr_tfidf, X_test_tfidf, test_data['stance'])

# save the model to disk
filename = './models/stance/NaiveBayes_tfidf.sav'
pickle.dump(CNB_clr, open(filename, 'wb'))

              precision    recall  f1-score   support

          -1       0.30      0.44      0.36        70
           0       0.30      0.63      0.41       126
           1       0.94      0.74      0.83       804

    accuracy                           0.71      1000
   macro avg       0.52      0.61      0.53      1000
weighted avg       0.82      0.71      0.74      1000

accuracy: 0.707


## 2.BoW

In [17]:
train_word_features, test_word_features, word_counter_vectorizer = get_word_count_features(train_data, test_data)

LinearSVC Classifire

In [18]:
clf = SVC(kernel="linear",class_weight="balanced", probability=True) #{-1:0.66, 0:0.29, 1:0.05}
clf.fit(train_word_features, train_data['stance'])
print_report(clf, test_word_features, test_data['stance'])

# save the model to disk
filename = './models/stance/LinearSVC_BoW.sav'
pickle.dump(clf, open(filename, 'wb'))

              precision    recall  f1-score   support

          -1       0.21      0.49      0.30        70
           0       0.29      0.54      0.38       126
           1       0.94      0.71      0.81       804

    accuracy                           0.68      1000
   macro avg       0.48      0.58      0.50      1000
weighted avg       0.81      0.68      0.72      1000

accuracy: 0.675


RandomForest Classifier

In [19]:
rf = RandomForestClassifier(class_weight="balanced")
rf_Bow = rf.fit(train_word_features, train_data['stance'])

print_report(rf_Bow, test_word_features, test_data['stance'])

# save the model to disk
filename = './models/stance/RandomForest_Bow.sav'
pickle.dump(rf_Bow, open(filename, 'wb'))

              precision    recall  f1-score   support

          -1       0.31      0.16      0.21        70
           0       0.44      0.32      0.37       126
           1       0.86      0.93      0.89       804

    accuracy                           0.80      1000
   macro avg       0.54      0.47      0.49      1000
weighted avg       0.77      0.80      0.78      1000

accuracy: 0.801


naive bayes

In [20]:
CNB_clr = ComplementNB()
CNB_clr_BoW = CNB_clr.fit(train_word_features, train_data['stance'])
print_report(CNB_clr_BoW,test_word_features, test_data['stance'])

# save the model to disk
filename = './models/stance/NaiveBayes_BoW.sav'
pickle.dump(CNB_clr, open(filename, 'wb'))

              precision    recall  f1-score   support

          -1       0.26      0.40      0.32        70
           0       0.30      0.67      0.42       126
           1       0.94      0.72      0.82       804

    accuracy                           0.69      1000
   macro avg       0.50      0.60      0.52      1000
weighted avg       0.82      0.69      0.73      1000

accuracy: 0.693


## 3.CBOW

In [21]:
X_train_vect_avg, X_test_vect_avg = get_word_embedding_features(train_data, test_data)

LinearSVC Classifier

In [22]:
clf = LinearSVC(class_weight="balanced") #{-1:0.66, 0:0.29, 1:0.05}
clf.fit(X_train_vect_avg, train_data['stance'])
print_report(clf, X_test_vect_avg, test_data['stance'])

# save the model to disk
filename = './models/stance/LinearSVC_CBOW.sav'
pickle.dump(clf, open(filename, 'wb'))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00        70
           0       0.00      0.00      0.00       126
           1       0.80      1.00      0.89       804

    accuracy                           0.80      1000
   macro avg       0.27      0.33      0.30      1000
weighted avg       0.65      0.80      0.72      1000

accuracy: 0.804


c:\Users\Nada\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nada\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nada\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


RandomForestClassifier

In [23]:
rf = RandomForestClassifier(class_weight="balanced")
rf_vect = rf.fit(X_train_vect_avg, train_data['stance'].values.ravel())
print_report(rf_vect, X_test_vect_avg, test_data['stance'])

# save the model to disk
filename = './models/stance/RandomForest_CBOW.sav'
pickle.dump(rf, open(filename, 'wb'))

              precision    recall  f1-score   support

          -1       0.36      0.06      0.10        70
           0       0.41      0.09      0.14       126
           1       0.82      0.98      0.89       804

    accuracy                           0.80      1000
   macro avg       0.53      0.37      0.38      1000
weighted avg       0.74      0.80      0.74      1000

accuracy: 0.803


## 4.Ara2Vec

In [24]:
train_data_embeddings = np.array([np.array([nlp(i).vector for i in ls if i in vocab]) for ls in train_data["text"]], dtype=object)
test_data_embeddings = np.array([np.array([nlp(i).vector for i in ls if i in vocab]) for ls in test_data["text"]], dtype=object)

In [25]:
X_train_vect_avg, X_test_vect_avg = avg_word_vector(train_data_embeddings, test_data_embeddings)

LinearSVC Classifier

In [26]:
clf = LinearSVC(class_weight={-1:0.66, 0:0.29, 1:0.05})
clf.fit(X_train_vect_avg, train_data['stance'])
# pipe_tfidf = make_pipeline(word_vectorizer, clf)
# pipe_tfidf.fit(train_data_embeddings, train_data['stance'])
print_report(clf, X_test_vect_avg, test_data['stance'])

# save the model to disk
filename = './models/stance/LinearSVC_Ara2Vec.sav'
pickle.dump(clf, open(filename, 'wb'))

              precision    recall  f1-score   support

          -1       0.12      0.50      0.19        70
           0       1.00      0.01      0.02       126
           1       0.85      0.74      0.79       804

    accuracy                           0.63      1000
   macro avg       0.65      0.42      0.33      1000
weighted avg       0.81      0.63      0.65      1000

accuracy: 0.632


c:\Users\Nada\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


RandomForestClassifier

In [27]:
rf = RandomForestClassifier(class_weight="balanced")
rf_vect = rf.fit(X_train_vect_avg, train_data['stance'].values.ravel())
print_report(rf_vect, X_test_vect_avg, test_data['stance'])

# save the model to disk
filename = './models/stance/RandomForest_Ara2Vec.sav'
pickle.dump(rf, open(filename, 'wb'))

              precision    recall  f1-score   support

          -1       0.36      0.06      0.10        70
           0       0.36      0.06      0.11       126
           1       0.82      0.98      0.89       804

    accuracy                           0.80      1000
   macro avg       0.51      0.37      0.37      1000
weighted avg       0.73      0.80      0.74      1000

accuracy: 0.802


## create initial embedding matrix 

In [28]:
weights_train_matrix = []
for word in vocab:
  weights_train_matrix.append(nlp(word).vector)

weights_train_matrix = torch.from_numpy(np.array(weights_train_matrix))
weights_train_matrix.size()

torch.Size([12538, 100])

# ArabicDataset
The class that impelements the dataset for arabic tweets

In [30]:
model = Classifier(weights_train_matrix)
print(model)

Classifier(
  (embedding): Embedding(12538, 100)
  (GRU): GRU(100, 100, batch_first=True)
  (linear): Linear(in_features=100, out_features=3, bias=True)
)


# Training

In [31]:
train_data_tokenized_as_num = train_data_tokenized.apply(lambda x: [word2index[word] for word in x])
# apply the same tokenization to the test set
test_data_tokenized_as_num = test_data_tokenized.apply(lambda x: [word2index[word] for word in x if word in word2index])
train_dataset = ArabicDataset(list(train_data_tokenized_as_num), train_data['stance'] + 1, word2index['<فراغ>'])
test_dataset = ArabicDataset(list(test_data_tokenized_as_num), test_data['stance'] + 1, word2index['<فراغ>'])

In [37]:
train(model, train_dataset, class_weights= torch.tensor([.5, .4, .1]), epochs=1, learning_rate=0.001)

100%|██████████| 219/219 [00:23<00:00,  9.29it/s]

Epochs: 1 | Train Loss: 0.1622123467272411         | Train Accuracy: 0.9397538900375366



In [34]:
# save the model to disk
filename = './models/stance/GRU_Ara2Vec.pth'
torch.save(model.state_dict(), filename)

# Evaluation

In [38]:
evaluate(model, test_dataset)

100%|██████████| 32/32 [00:00<00:00, 48.12it/s]


accuracy: 0.764
              precision    recall  f1-score   support

           0       0.26      0.33      0.29        70
           1       0.37      0.46      0.41       126
           2       0.90      0.85      0.88       804

    accuracy                           0.76      1000
   macro avg       0.51      0.55      0.53      1000
weighted avg       0.79      0.76      0.78      1000


Test Accuracy: 0.7639999985694885


array([[ 0.41529337,  0.735953  , -1.1096768 ],
       [-3.9900215 , -2.0137615 ,  5.4967074 ],
       [-4.189743  , -2.1851547 ,  5.6949573 ],
       ...,
       [-2.2286515 , -1.9069625 ,  3.7751677 ],
       [-0.6142367 , -1.37836   ,  1.9775208 ],
       [ 0.08092624, -0.49691126,  0.61754024]], dtype=float32)